# Modern NLP in Python
### _- Or -_
## What you can learn about food by analyzing a million Yelp reviews

## Outline
We are gonna be starting with **raw data** and running through **preparing**, **modeling**, **visualizing**, and **analyzing** the data. We'll touch on the following points:
1. A tour of the dataset
1. Introduction to text processing with spaCy
1. Automatic phrase modeling
1. Topic modeling with LDA
1. Visualizing topic models with pyLDAvis
1. Word vector models with word2vec
1. Visualizing word2vec with t-SNE

## The Yelp Dataset
* [**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. 
* It's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

Here's how to get the dataset:
1. Please visit the Yelp dataset webpage [here](https://www.yelp.com/dataset_challenge/)
1. Click "Get the Data"
1. Please review, agree to, and respect Yelp's terms of use!
1. The dataset downloads as a compressed .tgz file; uncompress it
1. Place the uncompressed dataset files (*yelp_academic_dataset_business.json*, etc.) in the directory named *material/yelp/source*

When focusing on restaurants alone, there are approximately __41K__ restaurants with approximately __1M__ user reviews written about them.

The data is provided in a handful of files in _.json_ format. We'll be using the following files for our demo:
- __yelp\_academic\_dataset\_business.json__ &mdash; _the records for individual businesses_
- __yelp\_academic\_dataset\_review.json__ &mdash; _the records for reviews users wrote about businesses_

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. Let's take a look at a few examples.

In [1]:
import os

data_directory = os.path.join('material', 'yelp', 'source')
businesses_filepath = os.path.join(data_directory, 'yelp_academic_dataset_business.json')

with open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id":"0DI8Dt2PJp07XkVvIElIcQ","name":"Innovative Vapors","neighborhood":"","address":"227 E Baseline Rd, Ste J2","city":"Tempe","state":"AZ","postal_code":"85283","latitude":33.3782141,"longitude":-111.936102,"stars":4.5,"review_count":17,"is_open":0,"attributes":["BikeParking: True","BusinessAcceptsBitcoin: False","BusinessAcceptsCreditCards: True","BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","DogsAllowed: False","RestaurantsPriceRange2: 2","WheelchairAccessible: True"],"categories":["Tobacco Shops","Nightlife","Vape Shops","Shopping"],"hours":["Monday 11:0-21:0","Tuesday 11:0-21:0","Wednesday 11:0-21:0","Thursday 11:0-21:0","Friday 11:0-22:0","Saturday 10:0-22:0","Sunday 11:0-18:0"],"type":"business"}



Only a few attributes will be of interest in task:
- __business\_id__ &mdash; _unique identifier for businesses_
- __categories__ &mdash; _an array containing relevant category values of businesses_

Moreover, we will focus von restaurant, which is indicated by the presence of the _Restaurant_ tag in the _categories_ array.

The review records are stored in a similar manner &mdash; _key, value_ pairs containing information about the reviews.

In [2]:
review_json_filepath = os.path.join(data_directory, 'yelp_academic_dataset_review.json')

with open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"NxL8SIC5yqOdnlXCg18IBg","user_id":"KpkOkG6RIf4Ra25Lhhxf1A","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2011-10-10","text":"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.","useful":0,"funny":0,"cool":0,"type":"review"}



The only attributes we a concerned with are:
- __business\_id__ &mdash; _indicates which business the review is about_
- __text__ &mdash; _the natural language text the user wrote_

_json_ is a handy file format for data interchange, but it's typically not the most usable for any sort of modeling work. Let's do a bit more data preparation to get our data in a more usable format. Our next code block will do the following:
1. Read in each business record and convert it to a Python `dict`
2. Filter out business records that aren't about restaurants (i.e., not in the "Restaurant" category)
3. Choose a subset of the restaurant to keep computations tractable.
4. Create a `frozenset` of the business IDs for restaurants, which we'll use in the next step

In [3]:
import json
from numpy.random import choice

restaurant_ids = set()

# open the businesses file
with open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
        if business['categories'] is None or 'Restaurants' not in business['categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business['business_id'])

# choose a subset of the restaurant ids
subset = []
subset_size = 20000
for restaurant_id, _ in zip(restaurant_ids, range(subset_size)):
    subset.append(restaurant_id)

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(subset)

# print the number of unique restaurant ids in the dataset
print('{} restaurants in the dataset'.format(len(restaurant_ids)))

20000 restaurants in the dataset


Next, we will create a new file that contains only the text from reviews about restaurants, with one review per line in the file.

In [4]:
intermediate_directory = os.path.join('material', 'yelp', 'intermediate_results')

review_txt_filepath = os.path.join(intermediate_directory, 'review_text_all.txt')

In [5]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    review_count = 0

    # create & open a new file in write mode
    with open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                #import pdb; pdb.set_trace()
                if review['business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review['text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print('Text from {} restaurant reviews written to the new txt file.'.format(review_count))
    
else:
    with open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass   
    print('Text from {} restaurant reviews in the txt file.'.format(review_count + 1))

Text from 1078371 restaurant reviews in the txt file.
CPU times: user 2.97 s, sys: 284 ms, total: 3.25 s
Wall time: 3.49 s


## spaCy &mdash; Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is a natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Sentence boundary detection
- Named entity recognition

spaCy is written in optimized Cython, which means it's _fast_.

In [6]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

Let's grab a sample review to play with.

In [7]:
with open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 2, 3))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Khotan sucks as a place to eat. The girls and I went to this place for "Dinner" but really all we got was appetizers. We went during happy hour so it was suppose to be cheaper, but it don't mean the size is bigger. I ordered some spicy tuna and it was $7 for like a tiny portion that didn't do anything for me at all. The roll were so tiny it was about the size of a quarter and I got four of those. BOO times 2. The only reason this place got two stars out of me was the bartender. They both were entertaining and he gave us some drinks for free. Atmosphere is really not impressive either.



Handing the review text to spaCy.

In [8]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.9 ms


In [9]:
print(parsed_review)

Khotan sucks as a place to eat. The girls and I went to this place for "Dinner" but really all we got was appetizers. We went during happy hour so it was suppose to be cheaper, but it don't mean the size is bigger. I ordered some spicy tuna and it was $7 for like a tiny portion that didn't do anything for me at all. The roll were so tiny it was about the size of a quarter and I got four of those. BOO times 2. The only reason this place got two stars out of me was the bartender. They both were entertaining and he gave us some drinks for free. Atmosphere is really not impressive either.



Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [10]:
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
Khotan sucks as a place to eat.

Sentence 2:
The girls and I went to this place for "Dinner" but really all we got was appetizers.

Sentence 3:
We went during happy hour

Sentence 4:
so it was suppose to be cheaper, but it don't mean the size is bigger.

Sentence 5:
I ordered some spicy tuna and it was $7 for like a tiny portion that didn't do anything for me at all.

Sentence 6:
The roll were so tiny it was about the size of a quarter and I got four of those.

Sentence 7:
BOO times 2.

Sentence 8:
The only reason this place got two stars out of me was the bartender.

Sentence 9:
They both were entertaining and he gave us some drinks for free.

Sentence 10:
Atmosphere is really not impressive either.




What about named entity detection?

In [11]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: Khotan - GPE

Entity 2: Dinner - PERSON

Entity 3: 7 - MONEY

Entity 4: four - CARDINAL

Entity 5: 2 - CARDINAL

Entity 6: two - CARDINAL



What about part of speech tagging?

In [12]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)), columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,Khotan,PROPN
1,sucks,VERB
2,as,ADP
3,a,DET
4,place,NOUN
5,to,PART
6,eat,VERB
7,.,PUNCT
8,The,DET
9,girls,NOUN


There is much more, like:
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [13]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: 'Yes' if x else ''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Khotan,-19.502029,,,,,Yes
1,sucks,-9.515271,,,,,
2,as,-5.534485,Yes,,,,
3,a,-3.929788,Yes,,,,
4,place,-7.954748,,,,,
5,to,-3.856022,Yes,,,,
6,eat,-8.822906,,,,,
7,.,-3.067898,,Yes,,,
8,The,-5.958707,Yes,,,,
9,girls,-9.163706,,,,,


## Phrase Modeling

_Phrase modeling_ is an approach to learning combinations of tokens that together represent meaningful multi-word concepts.  
We can develop phrase models by looping over the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance.  
The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A,\ B)}{count(A) * count(B)} > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A,\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text.  
When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token (so _new york_ would become *new\_york*). 

We will use the [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling.

In [14]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.models.phrases import Phraser

Slow version of gensim.models.doc2vec is being used


As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time:

1. Segment text of complete reviews into sentences.
1. Normalize text.
1. Phrase modeling $\rightarrow$ _apply phrase model to transform sentences_

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [15]:
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=4):
        for sent in parsed_review.sents:
            yield ' '.join([token.lemma_ for token in sent if not punct_space(token)])

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    with open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    return token.is_punct or token.is_space

In [16]:
unigram_sentences_filepath = os.path.join(intermediate_directory, 'unigram_sentences_all.txt')

We'll write the data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [17]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


If your data is organized like we just did gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components.  
It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once.  
This allows you to scale your modeling pipeline up to potentially very large corpora.

In [18]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [19]:
for unigram_sentence in it.islice(unigram_sentences, 100, 110):
    print(' '.join(unigram_sentence))
    print('---')

by the time we finish our large dinner order we be almost full but definitely not ready to stop eat
---
we each order another round of nigiri my boyfriend get tuna and i get salmon yum
---
what a hide gem thanks to groupon for help us find this one
---
we will definitely be return next time we 're in vegas
---
sushi be very average and very expensive with no wow factor
---
taste just bleh
---
miso soup be very salty and ginger salad be make from a lot of weird lettuce that be not cut small and difficult to eat
---
take a long time to get the salad
---
we ask to sit on the balcony to watch the show and they give us a table at the backside so we could not see
---
service be barely ok and keep make excuse why it be take so long
---


Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`rib eye`", to be linked together to form a new, single token: "`rib_eye`".

In [20]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [21]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if False:
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

# A Phraser is smaller and faster
bigram_phraser = Phraser(bigram_model)

CPU times: user 1min 30s, sys: 696 ms, total: 1min 31s
Wall time: 1min 31s


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [22]:
bigram_sentences_filepath = os.path.join(intermediate_directory, 'bigram_sentences_all.txt')

In [23]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = ' '.join(bigram_phraser[unigram_sentence])
            f.write(bigram_sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


In [24]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [25]:
for bigram_sentence in it.islice(bigram_sentences, 1480, 1500):
    print(' '.join(bigram_sentence))
    print('---')

it have go through a few name change since then but have be luna 's for the past several year
---
my really good experience here have outweigh the not so great experience
---
i consider it a local spot so it ' not five_star dining by any stretch
---
but the portion be still good sized and there be generally daily_special for a good price
---
i miss the old farmer_boy restaurant but i think luna 's be still a good option to consider if you 're look for a good meal for a reasonable price
---
bad than a grade_school cafeteria can food
---
only bright spot it be definitely low salt
---
french_toast be mushy and nasty
---
coffee smell burn but have no taste
---
ask non coffee drinking husband to try and he agree
---
no_matter the coupon not worth the meal
---
my wife and i both order the rib_eye steak dinner for $_10.99
---
it come with a choice of soup or salad we choose the salad with a delicious greek house dressing giant bake_potato and mixed_vegetable out of a can and free dessert choi

We now see two-word phrases, such as "`ice_cream`" and "`french_toast`", linked together in the text as a single token.  

Next, we'll train a _second-order_ phrase model. We'll apply the second-order phrase model on top of the already-transformed data, so that incomplete word combinations like "`rib eye steak`" will become fully joined to "`rib_eye_steak`".

In [26]:
trigram_model_filepath = os.path.join(intermediate_directory, 'trigram_model_all')

In [27]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if False:
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)

# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)
# A Phraser is smaller and faster
trigram_phraser = Phraser(trigram_model)

CPU times: user 2min 6s, sys: 628 ms, total: 2min 7s
Wall time: 2min 7s


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [28]:
trigram_sentences_filepath = os.path.join(intermediate_directory, 'trigram_sentences_all.txt')

In [29]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for bigram_sentence in bigram_sentences:
            trigram_sentence = ' '.join(trigram_phraser[bigram_sentence])
            f.write(trigram_sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [30]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [31]:
for trigram_sentence in it.islice(trigram_sentences, 1480, 1500):
    print(' '.join(trigram_sentence))
    print('---')

it have go through a few name change since then but have be luna_'s for the past several year
---
my really good experience here have outweigh the not so great experience
---
i consider it a local spot so it ' not five_star dining by any stretch
---
but the portion be still good sized and there be generally daily_special for a good price
---
i miss the old farmer_boy restaurant but i think luna_'s be still a good option to consider if you 're look for a good meal for a reasonable_price
---
bad than a grade_school_cafeteria can food
---
only bright spot it be definitely low salt
---
french_toast be mushy and nasty
---
coffee smell burn but have no taste
---
ask non coffee drinking husband to try and he agree
---
no_matter the coupon not worth the meal
---
my wife and i both order the rib_eye_steak dinner for $_10.99
---
it come with a choice of soup or salad we choose the salad with a delicious greek house dressing giant bake_potato and mixed_vegetable out of a can and free dessert choi

Looks like the second-order phrase model was successful. We're now seeing three-word phrases, such as "`rib_eye_steak`".

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [32]:
trigram_reviews_filepath = os.path.join(intermediate_directory, 'trigram_transformed_reviews_all.txt')

In [33]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_phraser[unigram_review]
            trigram_review = trigram_phraser[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = ' '.join(trigram_review)
            f.write(trigram_review + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.2 µs


Let's grab the same review from the file with the normalized and transformed text, and compare the two.

In [34]:
print('Original:\n')
for review in it.islice(line_review(review_txt_filepath), 49, 50):
    print(review)

print('----\n')

print('Transformed:\n')
with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 49, 50):
        print(review)

Original:

No no no no! Where do I start, perhaps slapping myself in the face for not having read the only other review on this place BEFORE crossing the threshold of the Clifton Fish Bar. 

To cut a long story short, I ordered a pizza with salami, peppers, mushrooms and anchovies. Simple. 45 mins later I received a pizza with peppers and anchovies. On quizzing the (grumpy) guy behind the counter as to the invisible ingredients i had ordered and paid for, he told me they didn't have any mushrooms or salami left. "Then why didn't you tell me this small, insignificant fact at the time i ordered, or subsequent 45 mins whilst I was in your shop?"...no reply other than a look of get outa my shop. Appalling. 

And how was the pizza you ask? Don't know, ask the rubbish bin not far from his shop.

If I could give 0 stars for this place, I would.

----

Transformed:

start slap face read review place cross threshold clifton fish bar cut long_story_short order pizza salami pepper mushroom anchov

Most of the grammatical structure has been removed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc.  
However, much of the general semantic *meaning* is still present.  
Also, multi-word concepts such as "`long_story_short`" and "`45_min`" have been joined into single tokens, as expected.  
The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented by a mixture of the individual tokens (words and phrases). In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](material/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [35]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import _pickle as pickle

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [36]:
trigram_dictionary_filepath = os.path.join(intermediate_directory, 'trigram_dict_all.dict')

In [37]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if False:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 36 ms, sys: 8 ms, total: 44 ms
Wall time: 61.9 ms


LDA uses the simplifying [*bag-of-words*](https://en.wikipedia.org/wiki/Bag-of-words_model) assumption.

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In [38]:
trigram_bow_filepath = os.path.join(intermediate_directory, 'trigram_bow_corpus_all.mm')

In [39]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [40]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if False:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 136 ms, sys: 4 ms, total: 140 ms
Wall time: 169 ms


Now we can learn our topic model from the reviews.  
We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to the `LdaMulticore` model.

In [41]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [42]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if False:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=4)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 308 ms, sys: 60 ms, total: 368 ms
Wall time: 706 ms


Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [43]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
    print('{:20} {}\n'.format('term', 'frequency'))
    for term, frequency in lda.show_topic(topic_number, topn=25):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

Iteresting topics are:  
0, 1, 8, 10, 11, 15, 17, 21, 23, 24, 28, 32, 35, 40, 42, 46, 48

In [44]:
explore_topic(topic_number=0)

term                 frequency

chicken              0.083
sauce                0.030
bbq                  0.023
wing                 0.021
meat                 0.021
try                  0.016
rib                  0.015
order                0.013
fried                0.012
tender               0.012
flavor               0.010
brisket              0.010
like                 0.010
eat                  0.009
fry                  0.009
best                 0.008
time                 0.008
spicy                0.008
come                 0.008
dry                  0.008
taste                0.007
juicy                0.006
sweet                0.006
pork                 0.006
pull_pork            0.006


Manually reviewing the top terms for each topic is a helpful exercise, but to get a deeper understanding of the topics and how they relate to each other, we need to visualize the data &mdash; preferably in an interactive format. Fortunately, we have the fantastic [**pyLDAvis**](https://pyldavis.readthedocs.io/en/latest/readme.html) library to help with that!

pyLDAvis includes a one-line function to take topic models created with gensim and prepare their data for visualization.

In [45]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [46]:
#%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [47]:
pyLDAvis.display(LDAvis_prepared)

### Wait, what am I looking at again?
There are a lot of moving parts in the visualization. Here's a brief summary:

* On the left, there is a plot of the "distance" between all of the topics (labeled as the _Intertopic Distance Map_)
  * The plot is rendered in two dimensions according a [*multidimensional scaling (MDS)*](https://en.wikipedia.org/wiki/Multidimensional_scaling) algorithm. Topics that are generally similar should appear close together on the plot, while *dis*similar topics should appear far apart.
  * The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.
  * An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.
* On the right, there is a bar chart showing top terms.
  * When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's *saliency* is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.
  * When a particular topic is selected, the bar chart changes to show the top-30 most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter $\lambda$, which can be adjusted with a slider above the bar chart.
    * Setting the $\lambda$ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.
    * Setting $\lambda$ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic &mdash; i.e., terms that occur *only* in this topic, and do not occur in other topics.
    * Setting $\lambda$ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.
* Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:
1. Better understanding and interpreting individual topics, and
1. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the $\lambda$ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the _Intertopic Distance Plot_ can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [54]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    return list(it.islice(line_review(review_txt_filepath), review_number, review_number+1))[0]

In [71]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_phraser[unigram_review]
    trigram_review = trigram_phraser[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.STOPWORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda x: -x[1])

    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        #print('{:25} {}'.format(topic_names[topic_number], round(freq, 3)))
        print("{}".format([word for word, frq in lda.show_topic(topic_number, topn=10)]))

In [81]:
sample_review = get_sample_review(256)
print(sample_review)

We used to enjoy Luna's for breakfast. 
The last 2 times the food has been bad. 
The main reason we won't go back, the last time we were in our waitress went to the wrong table, realized it then cussed. No big deal if my child wasn't with us. 
Poor service and lousy food. Great prices though. Hopefully it gets better.



In [82]:
lda_description(sample_review)

['order', 'wait', 'time', 'come', 'ask', 'service', 'table', 'server', 'tell', 'waitress']
['service', 'bad', 'price', 'better', 'quality', 'time', 'restaurant', 'star', 'like', 'location']
['breakfast', 'coffee', 'egg', 'brunch', 'pancake', 'come', 'bacon', 'great', 'like', 'wait']


## Word Vector Embedding with Word2Vec

Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

* The goal of *word vector embedding models*, is to learn dense, numerical vector representations for each term in a corpus vocabulary. 
* The vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents...
* ... and the relationship between it and other terms in the vocabulary. 
* Word vector models are also fully unsupervised

* The general idea of word2vec is, for a given *focus word*, to use the *context* of the word
* To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.
* At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. 
* The model then slides the window across every snippet of text in the corpus. 
* Each time the model considers a new snippet, it tries to learn some information about the focus word.
* Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [83]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [84]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if False:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print('{} training epochs so far.'.format(food2vec.train_count))

12 training epochs so far.
CPU times: user 1.24 s, sys: 84 ms, total: 1.32 s
Wall time: 1.65 s


In [85]:
print('{} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab)))

54471 terms in the food2vec vocabulary.


Let's take a look at the word vectors our model has learned.

In [87]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda x: -x[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
be,0.010989,-0.098580,-0.110538,0.105267,-0.055592,0.113280,-0.089227,0.036274,-0.119766,0.155558,...,-0.061357,-0.026129,0.023734,0.264967,0.034338,0.043833,-0.060932,0.059840,-0.052866,-0.110872
the,-0.083832,-0.152506,-0.020423,0.054192,-0.013097,0.022567,-0.182839,0.145732,-0.151383,0.007145,...,-0.010430,0.025187,-0.091061,0.111775,0.054993,0.033513,-0.015198,-0.052137,0.055071,-0.003255
and,-0.010588,-0.186269,0.010744,-0.009883,-0.038689,0.008270,-0.228434,0.087713,-0.054035,0.000333,...,0.111528,-0.095283,-0.047689,0.069654,0.166584,0.048733,0.000678,0.053589,-0.016841,-0.012481
i,-0.233326,-0.112178,-0.090333,0.197657,-0.068395,0.057032,-0.066095,-0.048882,-0.172856,0.022692,...,0.180020,0.048842,0.008773,-0.030269,-0.036047,0.095967,0.084133,-0.053520,-0.114601,-0.090002
a,-0.148915,-0.240049,-0.254629,0.107618,0.009426,0.007520,-0.173996,0.097633,-0.067905,0.067453,...,0.159938,0.060562,-0.074305,0.145551,-0.053099,-0.109160,0.147147,-0.073397,-0.015614,-0.033916
to,-0.147408,-0.144266,-0.072951,0.055676,-0.191244,0.128530,-0.186840,0.005135,-0.095664,0.256163,...,0.044601,0.067998,-0.027925,0.016943,0.036663,0.086091,-0.015894,0.026792,-0.146574,-0.026264
it,-0.126023,-0.229797,-0.063922,0.078058,0.104852,0.141732,-0.109144,0.143855,-0.003734,0.130395,...,0.245292,0.001967,-0.082077,0.083971,0.094040,0.090488,0.153816,-0.007391,0.004069,0.100732
have,-0.154819,-0.211079,-0.051330,-0.025575,-0.031188,0.018657,-0.139687,0.042059,-0.144194,-0.022015,...,-0.046991,-0.039086,-0.044456,0.081584,-0.029428,0.092095,0.018232,-0.001896,-0.121867,-0.032781
of,-0.095553,-0.040991,-0.030406,-0.054660,0.033540,0.123049,-0.114816,0.026980,0.099128,0.064965,...,-0.011258,-0.051014,-0.166438,0.218955,-0.106607,0.054742,-0.112908,0.036532,-0.131346,0.020330
not,-0.193577,-0.091389,-0.007253,0.235965,-0.186041,0.175037,-0.188715,-0.029255,-0.087940,-0.061813,...,0.043894,-0.090898,0.073228,-0.062479,0.047695,0.041620,0.060411,0.112908,-0.040006,0.094330


**What is the size of the wall of numbers?**

### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [92]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """
    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):
        print('{:20} {}'.format(word, round(similarity, 3)))

### What things are like Burger King?

In [99]:
get_related_terms('burger_king')

mcdonalds            0.931
mcdonald_'s          0.929
wendy_'s             0.907
mcd_'s               0.875
bk                   0.849
mcd                  0.834
mc_donald_'s         0.834
carl_'s_jr.          0.809
harvey_'s            0.795
five_guy             0.781


The model has learned that fast food restaurants are similar to each other! In particular, *mcdonalds* and *wendy's* are the most similar to Burger King, according to this dataset. In addition, the model has found that alternate spellings for the same entities are probably related, such as *mcdonalds*, *mcdonald's* and *mcd's*.

In [100]:
get_related_terms('soccer')

hockey               0.845
rugby                0.793
sport                0.787
football             0.775
steelers             0.763
espn                 0.755
basketball           0.742
soccer_game          0.736
football_game        0.708
olympics             0.705


In [101]:
get_related_terms('fork')

knife                0.913
butter_knife         0.819
spoon                0.797
steak_knife          0.783
chopstick            0.747
plastic_fork         0.741
plastic_knife        0.74
chop_stick           0.723
napkin               0.701
sharp_knife          0.693


In [103]:
get_related_terms('apple')

pear                 0.871
pecan                0.855
almond               0.842
apricot              0.842
peach                0.816
fig                  0.816
green_apple          0.802
blueberry            0.8
rhubarb              0.799
walnut               0.797


In [61]:
get_related_terms('happy_hour')

reverse_happy_hour   0.87
hh                   0.869
happy_hr             0.822
hh.                  0.818
during_happy_hour    0.789
taco_tuesday         0.732
m_f                  0.709
3pm-6pm              0.707
3_7pm                0.7
happy_hours          0.699


In [107]:
get_related_terms('tip')

gratuity             0.762
tax                  0.727
tab                  0.687
18_tip               0.686
bill                 0.681
$_120                0.679
include_tax          0.662
20_gratuity          0.661
$_100                0.658
$_150                0.657


In [62]:
get_related_terms('pasta', topn=20)

fettuccine           0.836
penne                0.835
lasagna              0.834
spaghetti            0.83
rigatoni             0.826
bolognese            0.822
lasagne              0.786
linguine             0.785
alfredo              0.785
tortellini           0.783
gnocchi              0.782
linguini             0.778
chicken_parm         0.775
manicotti            0.775
eggplant_parmesan    0.774
ravioli              0.773
penne_pasta          0.773
angel_hair           0.768
fettuccini           0.76
meatball             0.758


## Word algebra!
The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

But more generally, you can think of the vectors that represent each word as encoding some information about the *meaning* or *concepts* of the word. What happens when you ask the model to combine the meaning and concepts of words in new ways? Let's see.

In [109]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)    
    for term, similarity in answers:
        print(term)

In [110]:
word_algebra(add=['breakfast', 'lunch'])

brunch


In [111]:
word_algebra(add=['lunch', 'night'], subtract=['day'])

dinner


In [112]:
word_algebra(add=['taco', 'chinese'], subtract=['mexican'])

dumpling


In [113]:
word_algebra(add=['bun', 'mexican'], subtract=['american'])

tortilla


In [114]:
word_algebra(add=['coffee', 'snack'], subtract=['drink'])

pastry


In [115]:
word_algebra(add=['burger_king', 'pizza'])

pizza_hut


In [118]:
word_algebra(add=['wine', 'hops'], subtract=['grapes'])

craft_beer


## Word Vector Visualization with t-SNE

* [t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. 
* It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [120]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [121]:
tsne_input = word_vectors.drop(spacy.en.STOPWORDS, errors='ignore')
tsne_input = tsne_input.head(5000)

In [123]:
tsne_input.head(10)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
food,-0.076840,-0.160895,0.010319,0.153751,-0.045242,0.057135,-0.025237,0.148921,0.034291,0.257439,...,0.107853,-0.110773,-0.013110,0.126202,0.187992,0.071631,0.052972,0.078340,-0.134420,-0.081099
place,-0.128703,-0.228257,-0.257955,0.004890,0.027539,0.064053,0.045496,0.009080,-0.046814,0.016866,...,0.182027,0.024901,-0.048341,0.089956,0.026714,0.259203,0.062063,0.081858,-0.181257,-0.234439
good,-0.145629,0.017670,-0.063299,0.084828,0.027693,-0.047405,-0.080710,0.125334,-0.127993,0.050772,...,0.018127,0.003129,-0.002458,-0.163911,0.107887,0.023900,0.018278,0.168725,0.069398,-0.076847
order,-0.130352,-0.100845,0.036228,-0.016239,-0.062027,-0.017752,-0.071569,-0.159909,-0.061374,-0.005073,...,-0.044469,-0.034061,0.002668,0.052927,-0.008593,-0.077465,0.046232,-0.032159,0.092767,-0.032315
great,0.018282,-0.108175,-0.059079,-0.058285,0.093526,-0.042195,-0.156941,0.153408,-0.099760,0.008468,...,-0.052875,0.036250,-0.121445,-0.034261,0.180397,-0.051449,-0.040137,0.107300,-0.021731,-0.061563
like,-0.240547,-0.189648,-0.002598,-0.051824,0.100519,0.049585,-0.160006,-0.151981,-0.000614,0.004996,...,-0.140863,0.087466,0.102191,-0.040661,0.032067,-0.066908,-0.123889,0.154099,-0.122684,0.001002
',0.040838,-0.154406,-0.109304,0.101687,0.163430,0.241373,-0.011948,0.131574,-0.024694,0.196428,...,0.041378,0.056746,0.002592,0.164326,0.012275,0.004702,0.126416,0.007788,-0.002569,-0.029380
come,-0.018099,-0.082972,-0.102738,-0.066879,-0.114218,0.032960,-0.007437,-0.055022,-0.074295,0.198018,...,-0.008440,-0.082673,-0.033042,0.152236,0.020270,-0.171674,-0.082296,-0.063716,0.023801,-0.057307
service,0.005536,-0.202007,-0.218962,0.159426,-0.147472,0.168582,0.006629,0.116270,-0.164493,0.069136,...,-0.031283,-0.081114,-0.011484,0.163603,0.243680,0.192322,-0.034619,0.145869,-0.004494,0.010007
time,-0.077446,-0.132936,-0.180608,-0.003777,-0.127893,0.161376,-0.166147,0.076802,-0.032563,0.114033,...,0.069907,-0.008498,-0.054251,0.044493,-0.085471,0.084672,0.078771,-0.024323,-0.141198,-0.109735


In [124]:
tsne_filepath = os.path.join(intermediate_directory, 'tsne_model')
tsne_vectors_filepath = os.path.join(intermediate_directory, 'tsne_vectors.npy')

In [128]:
%%time

if False:
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=['x_coord', 'y_coord'])

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.49 ms


Now we have a two-dimensional representation of our data! Let's take a look.

In [129]:
tsne_vectors.head()

,x_coord,y_coord
food,-2.411427,-8.838213
place,-5.883048,8.128523
good,7.497086,2.369288
order,-2.360771,-4.572480
great,7.660513,3.280378


In [130]:
tsne_vectors['word'] = tsne_vectors.index

### Plotting with Bokeh

In [134]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

/home/user/projekte/anaconda3/envs/ipynb/lib/python3.6/site-packages/bokeh/core/json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Loading BokehJS ...

In [135]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom,'
                           'box_select, resize, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering

...and more generally are a powerful way machines can help humans make sense of what's in a giant pile of text. They're also often useful as a pre-processing step for many other downstream machine learning applications.